In [1]:
!pip install nltk
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
import numpy as np
import nltk
from nltk.stem import LancasterStemmer

# Khởi tạo stemmer
stemmer = LancasterStemmer()


In [ ]:

import nltk
nltk.download('punkt_tab')

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random
import json
import pickle

## 1. Tạo file intents.json
import json

# Lưu dữ liệu intents vào file intents.json
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Xin chào", "Chào bạn", "Hello", "Hi"],
            "responses": ["Xin chào! Tôi có thể giúp gì cho bạn?", "Chào bạn! Bạn cần hỗ trợ gì?"],
            "context_set": ""
        },
        {
            "tag": "product_recommend",
            "patterns": [
                "Tôi muốn chọn size quần áo",
                "Bạn có thể giúp tôi chọn size không?",
                "Tôi cần tìm size phù hợp"
            ],
            "responses": [
                "Tôi có thể giúp bạn chọn size và sản phẩm. Bạn có thể cho tôi biết giới tính, chiều cao, cân nặng và các thông tin khác không?",
                "Vui lòng cung cấp thông tin về giới tính, chiều cao, cân nặng để tôi tìm size phù hợp cho bạn."
            ],
            "context_set": "request_size_info"
        },
        {
            "tag": "missing_info_male",
            "patterns": [
                "Tôi là nam, cao 170cm, nặng 60kg",
                "Nam, 170cm, 60kg"
            ],
            "responses": [
                "Cảm ơn bạn đã cung cấp thông tin! Tôi sẽ tìm size phù hợp cho bạn ngay."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "missing_info_female",
            "patterns": [
                "Tôi là nữ, cao 160cm, nặng 50kg, vòng 1 là 85cm, vòng 2 là 65cm, vòng 3 là 90cm",
                "Nữ, cao 160cm, nặng 50kg, vòng 1 85, vòng 2 65, vòng 3 90"
            ],
            "responses": [
                "Cảm ơn bạn đã cung cấp thông tin! Tôi sẽ tìm size phù hợp cho bạn ngay."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "missing_info_request",
            "patterns": [
                "Tôi là nam",
                "Tôi là nữ",
                "Tôi cao 170cm",
                "Tôi nặng 60kg"
            ],
            "responses": [
                "Bạn có thể cung cấp thêm thông tin về chiều cao và cân nặng nếu là nam?",
                "Nếu bạn là nữ, vui lòng cung cấp thêm vòng 1, vòng 2, vòng 3 để tôi có thể hỗ trợ tốt hơn."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "goodbye",
            "patterns": ["Tạm biệt", "Hẹn gặp lại", "Chào nhé"],
            "responses": ["Hẹn gặp lại! Chúc bạn một ngày tốt lành!", "Tạm biệt! Hy vọng được gặp lại bạn."]
        }
    ]
}

with open('intents.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

## 2. Xử lý dữ liệu
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

# Đọc dữ liệu từ intents.json
with open('intents.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

            if intent["tag"] not in labels:
                labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# Xây dựng mô hình neural network sử dụng Keras
model = Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

# Biên dịch mô hình
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(training, output, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

# def chat():
#     print("Start talking with the bot! (type 'quit' to stop)")
#     inputs = ["Xin chào", "Tôi là nam, cao 170cm, nặng 60kg", "Tạm biệt"]
#     for inp in inputs:
#         print(f"You: {inp}")
#         if inp.lower() == "quit":
#             break

#         results = model.predict(np.array([bag_of_words(inp, words)]))
#         results_index = np.argmax(results)
#         tag = labels[results_index]

#         for tg in data["intents"]:
#             if tg["tag"] == tag:
#                 responses = tg['responses']

#         print(random.choice(responses))

# chat()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1889 - loss: 1.8888
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0956 - loss: 1.8639 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1644 - loss: 1.7694     
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2422 - loss: 1.7518 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2978 - loss: 1.5698 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5867 - loss: 1.5133 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6333 - loss: 1.4486  
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3778 - loss: 1.3772  
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5067 - loss: 1.5019  
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5667 - loss: 1.4022 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5422 - loss: 1.2777 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.640

Start talking with the bot! (type 'quit' to stop)
You: Xin chào
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Xin chào! Tôi có thể giúp gì cho bạn?
You: Tôi là nam, cao 170cm, nặng 60kg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Cảm ơn bạn đã cung cấp thông tin! Tôi sẽ tìm size phù hợp cho bạn ngay.
You: Tạm biệt
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Tạm biệt! Hy vọng được gặp lại bạn.


In [ ]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widget để nhập liệu
input_box = widgets.Text(placeholder="Type your message here...")
send_button = widgets.Button(description="Send")
output_box = widgets.Output()

# Hàm xử lý tin nhắn
def handle_message(change):
    user_input = input_box.value
    input_box.value = ""  # Xóa nội dung ô nhập liệu

    if user_input.lower() == "quit":
        with output_box:
            clear_output()
            print("Chatbot: Hẹn gặp lại! Chúc bạn một ngày tốt lành!")
        return

    # Dự đoán kết quả từ mô hình
    results = model.predict(np.array([bag_of_words(user_input, words)]))
    results_index = np.argmax(results)
    tag = labels[results_index]

    # Tìm phản hồi từ intents
    response = "Xin lỗi, tôi chưa hiểu câu hỏi của bạn."
    for tg in data["intents"]:
        if tg["tag"] == tag:
            response = random.choice(tg['responses'])
            break

    # Hiển thị phản hồi
    with output_box:
        print(f"You: {user_input}")
        print(f"Chatbot: {response}")

# Gắn hàm xử lý vào nút gửi
send_button.on_click(lambda x: handle_message(None))
input_box.on_submit(handle_message)

# Hiển thị giao diện
display(input_box, send_button, output_box)


Text(value='', placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())

Output()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [2]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
import numpy as np
import nltk
from nltk.stem import LancasterStemmer

# Khởi tạo stemmer
stemmer = LancasterStemmer()


In [3]:

import nltk
nltk.download('punkt_tab')

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random
import json
import pickle

## 1. Tạo file intents.json
import json

# Lưu dữ liệu intents vào file intents.json
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Xin chào", "Chào bạn", "Hello", "Hi"],
            "responses": ["Xin chào! Tôi có thể giúp gì cho bạn?", "Chào bạn! Bạn cần hỗ trợ gì?"],
            "context_set": ""
        },
        {
            "tag": "product_recommend",
            "patterns": [
                "Tôi muốn chọn size quần áo",
                "Bạn có thể giúp tôi chọn size không?",
                "Tôi cần tìm size phù hợp"
            ],
            "responses": [
                "Tôi có thể giúp bạn chọn size và sản phẩm. Bạn có thể cho tôi biết giới tính, chiều cao, cân nặng và các thông tin khác không?",
                "Vui lòng cung cấp thông tin về giới tính, chiều cao, cân nặng để tôi tìm size phù hợp cho bạn."
            ],
            "context_set": "request_size_info"
        },
        {
            "tag": "missing_info_male",
            "patterns": [
                "Tôi là nam, cao 170cm, nặng 60kg",
                "Nam, 170cm, 60kg"
            ],
            "responses": [
                "Cảm ơn bạn đã cung cấp thông tin! Tôi sẽ tìm size phù hợp cho bạn ngay."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "missing_info_female",
            "patterns": [
                "Tôi là nữ, cao 160cm, nặng 50kg, vòng 1 là 85cm, vòng 2 là 65cm, vòng 3 là 90cm",
                "Nữ, cao 160cm, nặng 50kg, vòng 1 85, vòng 2 65, vòng 3 90"
            ],
            "responses": [
                "Cảm ơn bạn đã cung cấp thông tin! Tôi sẽ tìm size phù hợp cho bạn ngay."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "missing_info_request",
            "patterns": [
                "Tôi là nam",
                "Tôi là nữ",
                "Tôi cao 170cm",
                "Tôi nặng 60kg"
            ],
            "responses": [
                "Bạn có thể cung cấp thêm thông tin về chiều cao và cân nặng nếu là nam?",
                "Nếu bạn là nữ, vui lòng cung cấp thêm vòng 1, vòng 2, vòng 3 để tôi có thể hỗ trợ tốt hơn."
            ],
            "context_filter": "request_size_info"
        },
        {
            "tag": "goodbye",
            "patterns": ["Tạm biệt", "Hẹn gặp lại", "Chào nhé"],
            "responses": ["Hẹn gặp lại! Chúc bạn một ngày tốt lành!", "Tạm biệt! Hy vọng được gặp lại bạn."]
        }
    ]
}

with open('intents.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)




# def chat():
#     print("Start talking with the bot! (type 'quit' to stop)")
#     inputs = ["Xin chào", "Tôi là nam, cao 170cm, nặng 60kg", "Tạm biệt"]
#     for inp in inputs:
#         print(f"You: {inp}")
#         if inp.lower() == "quit":
#             break

#         results = model.predict(np.array([bag_of_words(inp, words)]))
#         results_index = np.argmax(results)
#         tag = labels[results_index]

#         for tg in data["intents"]:
#             if tg["tag"] == tag:
#                 responses = tg['responses']

#         print(random.choice(responses))

# chat()


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
def add_intents_to_json(file_path, new_intents):
    try:
        # Đọc dữ liệu từ file JSON hiện có
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # Thêm các intent mới vào danh sách intents
        data['intents'].extend(new_intents)

        # Ghi lại dữ liệu vào file JSON
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

        print("Các intent mới đã được thêm thành công!")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")


def add_intent_to_array(arr_intent, new_intent):
    arr_intent.append(new_intent)
    return arr_intent


arr_intent = [
    {
        "tag": "thanks",
        "patterns": ["Cảm ơn", "Thanks", "Thank you"],
        "responses": ["Không có gì! Rất vui được giúp bạn.", "Bạn không cần khách sáo!"]
    },

    {
    "tag": "question",
    "patterns": [
    "Bạn là ai",
    "Bạn là gì?",
    "Ai đang nói với tôi?",
    "Tôi đang nói chuyện với ai vậy?",
    "Chào bạn",
    "Xin chào",
    "Kính chào",
    "Bạn đến từ đâu?",
    "Bạn có thể làm gì?"
                ],
    "responses": [
    "Tôi là hệ thống giao tiếp ảo của shop CuSan, Hãy cho biết hôm nay bạn muốn gì",
    "Chào bạn! Mình là trợ lý ảo của shop CuSan. Bạn cần mình giúp gì không?",
    "Mình là CuSan, bot của shop CuSan. Hôm nay bạn muốn tìm hiểu sản phẩm gì nào?",
    "Bạn đang tìm kiếm gì ở shop CuSan? Mình sẵn sàng hỗ trợ bạn!"
            ]
    }
]





In [5]:
# Tạo ra danh sách
def generate_range(min_value, max_value):
    return [i for i in range(min_value, max_value + 1)]

# Tạo ra mẫu chi tiết
def generate_detailed_patterns(height_range, weight_range, gender):
    patterns = []
    for height in height_range:
        for weight in weight_range:
            patterns.append(f"Chiều cao {height}cm, cân nặng {weight}kg và giới tính {gender}")
    return patterns

def generate(value, gender, check):
    patterns = []
    for i in value:
        if check:
            patterns.append(f"Chiều cao {i}cm và giới tính {gender}")
        else:
            patterns.append(f"Cân nặng {i}kg và giới tính {gender}")
    return patterns

# Hàm tạo intent mới
def create_intent(tag, height_min, height_max, weight_min, weight_max, gender, response):
    # Tạo khoảng giá trị cho chiều cao và cân nặng
    height_range = generate_range(height_min, height_max)
    weight_range = generate_range(weight_min, weight_max)

    # Tạo các mẫu câu hỏi
    patterns = generate(height_range, gender, True)
    patterns.extend(generate(weight_range, gender, False))
    patterns.extend(generate_detailed_patterns(height_range, weight_range, gender))

    # Thêm các câu trả lời
    response_patterns = ["Dựa trên thông tin bạn cung cấp, kích cỡ mà tôi đề xuất ", "Cảm ơn bạn đã cung cấp thông tin! Đây là kích thước phù hợp cho bạn", "Tôi nghĩ bạn nên sử dụng kích cỡ là "]
    res = []
    for i in response_patterns:
      respon = str(i) + response
      res.append(respon)

    # Tạo intent
    intent = {
        "tag": tag,
        "patterns": patterns,
        "responses": res
    }
    return intent


In [6]:
# Tạo một danh sách các thông tin
data = [
    {"tag": "size_info_famale", "height_min": 147, "height_max": 153, "weight_min": 38, "weight_max": 43, "gender": "nữ", "response": "Xs"},
    {"tag": "size_info_famale", "height_min": 150, "height_max": 155, "weight_min": 41, "weight_max": 46, "gender": "nữ", "response": "S"},
    {"tag": "size_info_famale", "height_min": 155  , "height_max": 163, "weight_min": 47, "weight_max": 52, "gender": "nữ", "response": "M"},
    {"tag": "size_info_famale", "height_min":160, "height_max":165, "weight_min": 53, "weight_max": 58, "gender": "nữ", "response": "L"},
      {"tag": "size_info_famale", "height_min":162, "height_max":166, "weight_min": 59, "weight_max": 64, "gender": "nữ", "response": "XL"},
    {"tag": "size_info_male", "height_min": 162, "height_max": 168, "weight_min": 57, "weight_max": 62, "gender": "nam", "response": "S"},
    {"tag": "size_info_male", "height_min": 169, "height_max": 173, "weight_min": 63, "weight_max": 67, "gender": "nam", "response": "M"},
    {"tag": "size_info_male", "height_min": 171  , "height_max": 175, "weight_min": 68, "weight_max": 72, "gender": "nam", "response": "L"},
    {"tag": "size_info_male", "height_min":173, "height_max":177, "weight_min": 73, "weight_max": 77, "gender": "nam", "response": "XL"},
      {"tag": "size_info_male", "height_min":175, "height_max":179, "weight_min": 78, "weight_max": 82, "gender": "nam", "response": "XXL"},

    {"tag": "size_info_child", "height_min": 95, "height_max": 101, "weight_min": 13, "weight_max": 15, "gender": "trẻ", "response": "2-2Y"},
    {"tag": "size_info_child", "height_min": 101, "height_max": 107, "weight_min": 15, "weight_max": 18, "gender": "trẻ", "response": "3-4Y"},
    {"tag": "size_info_child", "height_min": 107  , "height_max": 113, "weight_min": 18, "weight_max": 22, "gender": "trẻ", "response": "4-5Y"},
    {"tag": "size_info_child", "height_min":120, "height_max":125, "weight_min": 25, "weight_max": 28, "gender": "trẻ", "response": "7Y"},
      {"tag": "size_info_child", "height_min":114, "height_max":119, "weight_min": 23, "weight_max": 25, "gender": "trẻ", "response": "6Y"},

]

for res in data:
  arr_intent.append(create_intent(res['tag'], res['height_min'], res['height_max'], res['weight_min'], res['weight_max'], res['gender'], res['response']))



In [7]:
add_intents_to_json('intents.json', arr_intent)

Các intent mới đã được thêm thành công!


In [8]:

def print_json_file(file_path):
    try:
        # Đọc nội dung của file intents.json
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # In ra nội dung của file intents.json
        print(json.dumps(data, ensure_ascii=False, indent=4))
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# Ví dụ sử dụng
print_json_file('intents.json')


{
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Xin chào",
                "Chào bạn",
                "Hello",
                "Hi"
            ],
            "responses": [
                "Xin chào! Tôi có thể giúp gì cho bạn?",
                "Chào bạn! Bạn cần hỗ trợ gì?"
            ],
            "context_set": ""
        },
        {
            "tag": "product_recommend",
            "patterns": [
                "Tôi muốn chọn size quần áo",
                "Bạn có thể giúp tôi chọn size không?",
                "Tôi cần tìm size phù hợp"
            ],
            "responses": [
                "Tôi có thể giúp bạn chọn size và sản phẩm. Bạn có thể cho tôi biết giới tính, chiều cao, cân nặng và các thông tin khác không?",
                "Vui lòng cung cấp thông tin về giới tính, chiều cao, cân nặng để tôi tìm size phù hợp cho bạn."
            ],
            "context_set": "request_size_info"
        },
        {
     

In [9]:
  ## 2. Xử lý dữ liệu
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import json

# Đọc dữ liệu từ intents.json
with open('intents.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

            if intent["tag"] not in labels:
                labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# Xây dựng mô hình neural network sử dụng Keras
model = Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation='softmax'))

# Biên dịch mô hình
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(training, output, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4437 - loss: 1.5187
Epoch 2/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9528 - loss: 0.2076
Epoch 3/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9458 - loss: 0.2233
Epoch 4/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9617 - loss: 0.1343
Epoch 5/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9700 - loss: 0.1096
Epoch 6/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9733 - loss: 0.1017
Epoch 7/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9711 - loss: 0.0881
Epoch 8/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9668 - loss: 0.0907
Epoch 9/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9692 - loss: 0.0969
Epoch 10/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9620 - loss: 0.1095
Epoch 11/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9628 - loss: 0.0956
Epoch 12/200
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

In [10]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00


In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widget để nhập liệu
input_box = widgets.Text(placeholder="Type your message here...")
send_button = widgets.Button(description="Send")
output_box = widgets.Output()

# Hàm xử lý tin nhắn
def handle_message(change):
    user_input = input_box.value
    input_box.value = ""  # Xóa nội dung ô nhập liệu

    if user_input.lower() == "quit":
        with output_box:
            clear_output()
            print("Chatbot: Hẹn gặp lại! Chúc bạn một ngày tốt lành!")
        return

    # Dự đoán kết quả từ mô hình
    results = model.predict(np.array([bag_of_words(user_input, words)]))
    results_index = np.argmax(results)
    tag = labels[results_index]

    # Tìm phản hồi từ intents
    response = "Xin lỗi, tôi chưa hiểu câu hỏi của bạn."
    for tg in data["intents"]:
        if tg["tag"] == tag:
            response = random.choice(tg['responses'])
            break

    # Hiển thị phản hồi
    with output_box:
        print(f"You: {user_input}")
        print(f"Chatbot: {response}")

# Gắn hàm xử lý vào nút gửi
send_button.on_click(lambda x: handle_message(None))
input_box.on_submit(handle_message)

# Hiển thị giao diện
display(input_box, send_button, output_box)


Text(value='', placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())

Output()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
